# Word embeddings and RNNs

## 1. The data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential 
from keras.layers import Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Dense, Dropout
from keras import preprocessing

df = pd.read_csv('Bank_complaints.csv')
df.head()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,Product,Consumer complaint narrative
0,Student loan,In XX/XX/XXXX I filled out the Fedlaon applica...
1,Student loan,I am being contacted by a debt collector for p...
2,Student loan,I cosigned XXXX student loans at SallieMae for...
3,Student loan,Navient has sytematically and illegally failed...
4,Student loan,My wife became eligible for XXXX Loan Forgiven...


In [2]:
import random
random.seed(123)
df = df.sample(2000)

In [3]:
df.index = range(2000)

In [4]:
product = df["Product"]

There are 7 types of complaints

In [5]:
product.value_counts()

Student loan                   403
Credit card                    340
Consumer Loan                  320
Mortgage                       265
Bank account or service        241
Credit reporting               226
Checking or savings account    205
Name: Product, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
Product                         2000 non-null object
Consumer complaint narrative    2000 non-null object
dtypes: object(2)
memory usage: 31.3+ KB


In [7]:
complaints = df["Consumer complaint narrative"]

## 2. Classifying bank complaints using word embeddings

### 2.1 Pretraining your own embedding layer

We're going to create our own embedding. Let's start with importing an embedding layer. 
- `input_dim`: We specify the vocabulary `n_words` to be equal to 1000.
- `output_dim`: The size of the vector space of the embedding vector (100)
- `input_length`: This the length of the sentences, so equal to `max_sentence_length` in our example.

In [8]:
n_words = 1000
output_dim = 100
max_sentence_length = 200 

embedding_layer = Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length)

Only keep 1,000 most common words and use one-hot encoding

In [9]:
tokenizer = Tokenizer(num_words = n_words)

In [10]:
tokenizer.fit_on_texts(complaints)

In [11]:
complaints_encoded = tokenizer.texts_to_sequences(complaints)

In [12]:
# this will turn the sequence array into an array with
# the 30 most common words per sequence.
seq_padded = preprocessing.sequence.pad_sequences(complaints_encoded, 
                                               maxlen=max_sentence_length)

In [13]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(product)
list(le.classes_)
product_cat = le.transform(product)
product_onehot = to_categorical(product_cat)

In [14]:
np.shape(product_onehot)

(2000, 7)

In [15]:
product_onehot

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [16]:
np.shape(seq_padded)

(2000, 200)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(seq_padded,  
            product_onehot, test_size=0.10, random_state=1234)

In [18]:
np.shape(x_train)

(1800, 200)

In [19]:
np.shape(x_test)

(200, 200)

In [20]:
np.shape(y_train)

(1800, 7)

In [21]:
np.shape(y_test)

(200, 7)

In [22]:
product_onehot.shape

(2000, 7)

In [23]:
from keras import regularizers

model = Sequential()
model.add(Embedding(n_words, output_dim, input_length = max_sentence_length))
model.add(Flatten())
model.add(Dense(50, activation='relu')) #input_shape=(20000,)
model.add(Dense(25, activation='relu'))

model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1000050   
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 182       
Total params: 1,101,507
Trainable params: 1,101,507
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(x_train, y_train, epochs=50, batch_size=256, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/50
1440/1440 [==============================] - 1s 503us/step - loss: 1.9453 - acc: 0.1590 - val_loss: 1.9260 - val_acc: 0.1833
Epoch 2/50
1440/1440 [==============================] - 0s 323us/step - loss: 1.7980 - acc: 0.3104 - val_loss: 1.9043 - val_acc: 0.2056
Epoch 3/50
1440/1440 [==============================] - 0s 320us/step - loss: 1.5779 - acc: 0.4819 - val_loss: 1.8629 - val_acc: 0.2028
Epoch 4/50
1440/1440 [==============================] - 0s 313us/step - loss: 1.3177 - acc: 0.6194 - val_loss: 1.8282 - val_acc: 0.3000
Epoch 5/50
1440/1440 [==============================] - 0s 311us/step - loss: 1.0319 - acc: 0.7271 - val_loss: 1.8664 - val_acc: 0.2000
Epoch 6/50
1440/1440 [==============================] - 0s 319us/step - loss: 0.8189 - acc: 0.8014 - val_loss: 1.7896 - val_acc: 0.3250
Epoch 7/50
1440/1440 [==============================] - 0s 331us/step - loss: 0.5985 - acc: 0.8826 - val_loss: 1.7172 - val_acc: 0.3472
E

In [25]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [26]:
loss

3.0200108528137206

In [27]:
accuracy

0.385

### 2.2 Using dropout regularization

In [28]:
from keras import regularizers

model = Sequential()
model.add(Embedding(n_words, output_dim, input_length = max_sentence_length))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(x_train, y_train, epochs=50, batch_size=256, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                1000050   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 182       
Total para

In [29]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [30]:
loss

2.653766975402832

In [31]:
accuracy

0.41

### 2.3 A pre-trained embedding layer with a deeper network

In [32]:
embed_index = {}
embedding_file = open('glove.6B.100d.txt')
for line in embedding_file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embed_index[word] = coefs
embedding_file.close()
len(embed_index)

400000

In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(complaints)
n_words = len(tokenizer.word_index) + 1

In [34]:
complaints_encoded = tokenizer.texts_to_sequences(complaints)

In [35]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((n_words, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embed_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [36]:
embedding_matrix.shape

(10555, 100)

In [37]:
max_sentence_length = 200
from keras.preprocessing.sequence import pad_sequences

padded_complaints= pad_sequences(complaints_encoded, maxlen=max_sentence_length)

In [38]:
# define model
model = Sequential()
model.add(Embedding(n_words, 100, weights=[embedding_matrix], input_length= 200, trainable=False))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))
# compile the model
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
history = model.fit(x_train, y_train, epochs=100, batch_size=256, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1055500   
_________________________________________________________________
flatten_3 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                1000050   
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_4 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 182       
Total para

Epoch 51/100
1440/1440 [==============================] - 0s 280us/step - loss: 0.6505 - acc: 0.7778 - val_loss: 1.9596 - val_acc: 0.2750
Epoch 52/100
1440/1440 [==============================] - 0s 269us/step - loss: 0.6710 - acc: 0.7667 - val_loss: 1.9346 - val_acc: 0.3083
Epoch 53/100
1440/1440 [==============================] - 0s 265us/step - loss: 0.6784 - acc: 0.7632 - val_loss: 1.9507 - val_acc: 0.3000
Epoch 54/100
1440/1440 [==============================] - 0s 270us/step - loss: 0.6114 - acc: 0.8000 - val_loss: 2.1946 - val_acc: 0.3111
Epoch 55/100
1440/1440 [==============================] - 0s 271us/step - loss: 0.6455 - acc: 0.7688 - val_loss: 2.1086 - val_acc: 0.3028
Epoch 56/100
1440/1440 [==============================] - 0s 269us/step - loss: 0.6106 - acc: 0.7847 - val_loss: 2.0042 - val_acc: 0.3306
Epoch 57/100
1440/1440 [==============================] - 0s 267us/step - loss: 0.6192 - acc: 0.7882 - val_loss: 2.2606 - val_acc: 0.2806
Epoch 58/100
1440/1440 [==========

In [39]:
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [40]:
loss

2.8106768894195557

In [41]:
accuracy

0.31

## 2. Classifying bank complaints using word embeddings and an RNN

### 2.1 A naive recurrent neural network

In [49]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

n_words = 1000
output_dim = 100
max_sentence_length = 200 

model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length))
model.add(SimpleRNN(100))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100)               20100     
Total params: 120,100
Trainable params: 120,100
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_12 (Dense)             (None, 7)                 707       
Total params: 120,807
Trainable params: 120,807
Non-trainable params: 0
_________________________________________________________________


In [52]:
history = model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.6080 - acc: 0.9375 - val_loss: 2.1563 - val_acc: 0.1667
Epoch 2/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.5096 - acc: 0.9493 - val_loss: 2.1201 - val_acc: 0.1861
Epoch 3/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.3177 - acc: 0.9854 - val_loss: 2.2764 - val_acc: 0.1944
Epoch 4/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.2703 - acc: 0.9868 - val_loss: 2.2984 - val_acc: 0.2167
Epoch 5/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.2977 - acc: 0.9701 - val_loss: 2.3413 - val_acc: 0.2000
Epoch 6/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.1293 - acc: 0.9951 - val_loss: 2.3718 - val_acc: 0.2028
Epoch 7/100
1440/1440 [==============================] - 2s 2ms/step - loss: 0.0791 - acc: 0.9979 - val_loss: 2.5378 - val_acc: 0.1806
Epoch 8/

1440/1440 [==============================] - 3s 2ms/step - loss: 0.0078 - acc: 0.9958 - val_loss: 4.9327 - val_acc: 0.1500
Epoch 61/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.4477 - acc: 0.8507 - val_loss: 4.1989 - val_acc: 0.2139
Epoch 62/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0208 - acc: 0.9944 - val_loss: 4.2183 - val_acc: 0.2111
Epoch 63/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0027 - acc: 1.0000 - val_loss: 4.1959 - val_acc: 0.1944
Epoch 64/100
1440/1440 [==============================] - 3s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 4.1344 - val_acc: 0.2083
Epoch 65/100
1440/1440 [==============================] - 3s 2ms/step - loss: 7.9136e-04 - acc: 1.0000 - val_loss: 4.1722 - val_acc: 0.2000
Epoch 66/100
1440/1440 [==============================] - 3s 2ms/step - loss: 5.2353e-04 - acc: 1.0000 - val_loss: 4.1848 - val_acc: 0.2111
Epoch 67/100
1440/1440 [=============================

In [55]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

n_words = 1000
output_dim = 100
max_sentence_length = 200 

model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          100000    
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 200, 100)          20100     
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 200, 100)          20100     
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 100)               20100     
Total params: 160,300
Trainable params: 160,300
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          100000    
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 200, 100)          20100     
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 200, 100)          20100     
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 100)               20100     
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 707       
Total params: 161,007
Trainable params: 161,007
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = model.fit(x_train, y_train, epochs=30, batch_size=128, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/30
1440/1440 [==============================] - 7s 5ms/step - loss: 3.2430e-05 - acc: 1.0000 - val_loss: 6.6745 - val_acc: 0.1556
Epoch 2/30
1440/1440 [==============================] - 7s 5ms/step - loss: 2.0631e-05 - acc: 1.0000 - val_loss: 6.8496 - val_acc: 0.1556
Epoch 3/30
1440/1440 [==============================] - 7s 5ms/step - loss: 1.2625e-05 - acc: 1.0000 - val_loss: 7.0048 - val_acc: 0.1556
Epoch 4/30
1440/1440 [==============================] - 7s 5ms/step - loss: 7.6063e-06 - acc: 1.0000 - val_loss: 7.1973 - val_acc: 0.1583
Epoch 5/30
1440/1440 [==============================] - 7s 5ms/step - loss: 5.3649e-06 - acc: 1.0000 - val_loss: 7.4218 - val_acc: 0.1556
Epoch 6/30
1440/1440 [==============================] - 7s 5ms/step - loss: 2.8226e-06 - acc: 1.0000 - val_loss: 7.5816 - val_acc: 0.1556
Epoch 7/30
1440/1440 [==============================] - 8s 5ms/step - loss: 1.6924e-06 - acc: 1.0000 - val_loss: 7.7757 - val

very poor result! A random classifier would perform similarly!

### 2.2 LSTM

In [61]:
from keras.layers import LSTM
model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length))
model.add(LSTM(100))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train, epochs=80, batch_size=128, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/80
1440/1440 [==============================] - 9s 6ms/step - loss: 1.9429 - acc: 0.1868 - val_loss: 1.9117 - val_acc: 0.2528
Epoch 2/80
1440/1440 [==============================] - 9s 6ms/step - loss: 1.8942 - acc: 0.2104 - val_loss: 1.8804 - val_acc: 0.2611
Epoch 3/80
1440/1440 [==============================] - 8s 6ms/step - loss: 1.8684 - acc: 0.2924 - val_loss: 1.8548 - val_acc: 0.2694
Epoch 4/80
1440/1440 [==============================] - 6s 4ms/step - loss: 1.8255 - acc: 0.3194 - val_loss: 1.7098 - val_acc: 0.3000
Epoch 5/80
1440/1440 [==============================] - 6s 4ms/step - loss: 1.6553 - acc: 0.3417 - val_loss: 1.7953 - val_acc: 0.2806
Epoch 6/80
1440/1440 [==============================] - 6s 4ms/step - loss: 1.5982 - acc: 0.3514 - val_loss: 1.5798 - val_acc: 0.3472
Epoch 7/80
1440/1440 [==============================] - 6s 4ms/step - loss: 1.5768 - acc: 0.3882 - val_loss: 1.5183 - val_acc: 0.3750
Epoch 8/80
1440

1440/1440 [==============================] - 9s 6ms/step - loss: 0.2227 - acc: 0.9618 - val_loss: 1.5737 - val_acc: 0.6472
Epoch 62/80
1440/1440 [==============================] - 9s 6ms/step - loss: 0.0567 - acc: 0.9861 - val_loss: 1.6857 - val_acc: 0.6000
Epoch 63/80
1440/1440 [==============================] - 8s 6ms/step - loss: 0.0762 - acc: 0.9806 - val_loss: 1.8288 - val_acc: 0.5833
Epoch 64/80
1440/1440 [==============================] - 7s 5ms/step - loss: 0.4341 - acc: 0.9188 - val_loss: 1.8154 - val_acc: 0.5389
Epoch 65/80
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0708 - acc: 0.9840 - val_loss: 1.6436 - val_acc: 0.5778
Epoch 66/80
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0309 - acc: 0.9986 - val_loss: 1.6363 - val_acc: 0.5944
Epoch 67/80
1440/1440 [==============================] - 7s 5ms/step - loss: 0.0226 - acc: 0.9993 - val_loss: 1.6593 - val_acc: 0.6111
Epoch 68/80
1440/1440 [==============================] - 8s 5ms/ste

In [62]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [63]:
loss

2.17295569896698

In [64]:
accuracy

0.535

### 2.3 GRU

In [76]:
from keras.layers import LSTM
model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train, epochs=40, batch_size=128, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/40
1440/1440 [==============================] - 27s 19ms/step - loss: 1.9287 - acc: 0.1889 - val_loss: 1.9076 - val_acc: 0.2444
Epoch 2/40
1440/1440 [==============================] - 21s 15ms/step - loss: 1.8037 - acc: 0.3021 - val_loss: 1.7238 - val_acc: 0.3083
Epoch 3/40
1440/1440 [==============================] - 23s 16ms/step - loss: 1.6053 - acc: 0.3687 - val_loss: 1.5926 - val_acc: 0.3833
Epoch 4/40
1440/1440 [==============================] - 23s 16ms/step - loss: 1.4751 - acc: 0.3972 - val_loss: 1.5985 - val_acc: 0.3722
Epoch 5/40
1440/1440 [==============================] - 22s 15ms/step - loss: 1.3497 - acc: 0.4347 - val_loss: 1.5836 - val_acc: 0.3944
Epoch 6/40
1440/1440 [==============================] - 22s 15ms/step - loss: 1.2750 - acc: 0.4840 - val_loss: 1.6237 - val_acc: 0.4056
Epoch 7/40
1440/1440 [==============================] - 22s 15ms/step - loss: 1.2175 - acc: 0.5104 - val_loss: 1.5345 - val_acc: 0.4500
E

In [68]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [69]:
loss

3.1009898948669434

In [70]:
accuracy

0.46

In [72]:
from keras.layers import GRU
model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 100, input_length = max_sentence_length))
model.add(GRU(100))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=50, batch_size=128, validation_split=0.2)

Train on 1440 samples, validate on 360 samples
Epoch 1/50
1440/1440 [==============================] - 8s 6ms/step - loss: 1.9319 - acc: 0.1882 - val_loss: 1.9091 - val_acc: 0.2306
Epoch 2/50
1440/1440 [==============================] - 6s 4ms/step - loss: 1.8868 - acc: 0.2069 - val_loss: 1.8800 - val_acc: 0.2389
Epoch 3/50
1440/1440 [==============================] - 6s 4ms/step - loss: 2.0737 - acc: 0.2868 - val_loss: 2.9031 - val_acc: 0.2722
Epoch 4/50
1440/1440 [==============================] - 7s 5ms/step - loss: 1.8483 - acc: 0.3972 - val_loss: 1.7770 - val_acc: 0.3167
Epoch 5/50
1440/1440 [==============================] - 6s 4ms/step - loss: 1.6678 - acc: 0.4167 - val_loss: 1.6581 - val_acc: 0.3250
Epoch 6/50
1440/1440 [==============================] - 6s 4ms/step - loss: 1.6608 - acc: 0.4236 - val_loss: 1.7790 - val_acc: 0.2972
Epoch 7/50
1440/1440 [==============================] - 7s 5ms/step - loss: 1.5249 - acc: 0.4653 - val_loss: 2.8480 - val_acc: 0.2833
Epoch 8/50
1440

In [73]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

In [74]:
loss

2.942488350868225

In [75]:
accuracy

0.455

# SOURCES

https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb

https://catalog.data.gov/dataset/consumer-complaint-database